# Project 2: Backtracking and Heavy Ball

In [3]:
import numpy as np
from numpy.linalg import norm
from scipy.optimize import minimize_scalar

### Task 1: Implement backtracking

In [4]:
def backtracking(f,grad_f,pk,xk,c1,rho):
    alpha = 1
    while f(xk + alpha*pk) > f(xk) + c1*alpha*np.dot(pk.T,grad_f(xk)):
        alpha = rho*alpha
    return alpha

### Task 2: Backtracking Steepest Descent Method

In [5]:
# Setting up objective, gradient, and Hessian functions

f = lambda x: (x[0]+2*x[1]-7)**2 + (2*x[0]+x[1]-5)**2
grad_f = lambda x: np.array([10*x[0]+8*x[1]-34, 8*x[0]+10*x[1]-38])

x0 = np.array([0.,0.])
x_opt = np.array([1, 3])

In [6]:
# Inexact line search with backtracking implemented

def backtrack_steepest_descent(x0,rho):     # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_f(x)                          # find descent direction
    i = 0                                   # starting counter for iteration
    while norm(p) > 1e-9:                   # if the norm is not small
        alpha = backtracking(f,grad_f,p,x,1e-4,rho)
                                            # Using backtracking alpha
        x = x + alpha * p                   # locate the next iterate
        p = -grad_f(x)                      # find next descent direction
        i += 1
    print(i,"Iteration")
    return x

print("Using Backtracking Inexact Steepest Descent")
print(backtrack_steepest_descent(x0,0.1))

Using Backtracking Inexact Steepest Descent
111 Iteration
[1. 3.]


In [7]:
# From Project 1

def exact_steepest_descent(x0):             # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_f(x)                          # find descent direction
    i = 0                                   # starting counter for iteration
    while norm(p) > 1e-9:                   # if the norm is not small
        def subproblem1d(alpha):            # define a 1d subproblem 
            return f(x + alpha * p)  
                                            # use minimize_scalar function
        res = minimize_scalar(subproblem1d) # to solve the subproblem
        x = x + res.x * p                   # locate the next iterate
        p = -grad_f(x)                      # find next descent direction
        i += 1
    print(i,"Iteration")
    return x

def inexact_steepest_descent(x0):           # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_f(x)                          # find descent direction
    i = 0                                   # starting counter for iteration
    while norm(p) > 1e-9:                   # if the norm is not small
        x = x + 1e-3 * p                    # locate the next iterate
        p = -grad_f(x)                      # find next descent direction
        i += 1
    print(i,"Iteration")
    return x

print("Using Exact Steepest Descent:")
print(exact_steepest_descent(x0))
print()
print("Using Inexact Steepest Descent:")
print(inexact_steepest_descent(x0))

Using Exact Steepest Descent:
13 Iteration
[1. 3.]

Using Inexact Steepest Descent:
10871 Iteration
[1. 3.]


In [8]:
print(backtrack_steepest_descent(x0,0.5)) # when rho = 0.5
print(backtrack_steepest_descent(x0,0.9)) # when rho = 0.9

55 Iteration
[1. 3.]
798 Iteration
[1. 3.]


### Task 3: Backtracking Newton's Method

In [9]:
# Setting up Rosenbrock and its gradient function

f = lambda x: (1-x[0])**2+100*(x[1]-x[0]**2)**2

grad_f = lambda x: np.array([400*x[0]**3-400*x[0]*x[1]+2*x[0]-2, 200*x[1]-200*x[0]**2])

hess_f = lambda x: np.matrix([[1200*x[0]**2-400*x[1]+2, -400*x[0]], [-400*x[0], 200]])                              

In [10]:
def newton_method(x0):                      # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_f(x)                          # find descent direction
    h = hess_f(x)                           # find hessian matrix
    i = 0
    while norm(p) > 1e-9:                   # if the norm is not small
        alpha = backtracking(f,grad_f,p,x,1e-4,0.9)
                                            # Using backtracking alpha
        newton_dir = np.linalg.solve(h, p)  # Newton direction
        x = x + alpha*newton_dir            # locate the next iterate
        p = -grad_f(x)                      # find next descent direction
        h = hess_f(x)                       # find next hessian matrix
        i += 1
    print(i,"Iteration")   
    return x

In [11]:
# When starting point is (1.2, 1.2)
x0 = np.array([1.2,1.2])

print("Using Backtracking Newton's Method")
print(newton_method(x0))
print()
print("Using Backtracking Inexact Steepest Descent")
print(backtrack_steepest_descent(x0,0.1))

Using Backtracking Newton's Method
13734 Iteration
[1. 1.]

Using Backtracking Inexact Steepest Descent
264 Iteration
[1. 1.]


In [12]:
# When starting point is (-1.2, 1)
x0 = np.array([-1.2,1])

print("Using Backtracking Newton's Method")
print(newton_method(x0))
print()
print("Using Backtracking Inexact Steepest Descent")
print(backtrack_steepest_descent(x0,0.1))

Using Backtracking Newton's Method
5644 Iteration
[1. 1.]

Using Backtracking Inexact Steepest Descent
946 Iteration
[1. 1.]


### Task 4: Heavy ball Method

In [13]:
def heavy_ball(x0):           # input is the starting point
    p = -grad_f(x0)                          # find descent direction
    x = x0 + 1e-3 * p                       # when k = 0 (first point)
    i = 1                                   # starting counter for iteration
    x_step = np.array([x0,x])               # to store step xk - xk-1
    while norm(p) > 1e-9:                   # if the norm is not small
        x = x + 1e-3 * p + 0.9*(x_step[1] - x_step[0])      # locate the next iterate
        x_step[0] = x_step[1]
        x_step[1] = x
        p = -grad_f(x)                      # find next descent direction
        i += 1
    print(i,"Iteration")
    return x      

In [14]:
print("Using Heavy ball Method")
print(heavy_ball(x0))
print()
print("Using Exact Steepest Descent:")
print(exact_steepest_descent(x0))
print()
print("Using Inexact Steepest Descent:")
print(inexact_steepest_descent(x0))

Using Heavy ball Method
4686 Iteration
[1. 1.]

Using Exact Steepest Descent:
24425 Iteration
[1. 1.]

Using Inexact Steepest Descent:
49370 Iteration
[1. 1.]
